### ***Import Necessary Modules***

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns

In [ ]:
data_train=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
data_train.head()

In [ ]:
data_train.shape

In [ ]:
data_test=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
data_test.head()

In [ ]:
data_train.describe()

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train.info()

In [ ]:
data_train=data_train.drop(columns=['Id','Alley','PoolQC','Fence','MiscFeature'])

### ***List Of Columns Having NULL Values***

In [ ]:
cols=data_train.columns[data_train.isnull().any()].tolist()
cols

In [ ]:
df1=pd.DataFrame()
df2=pd.DataFrame()
l=[]
r=[]
for i in data_train:
  if(np.dtype(data_train[i])=='object'):
    l.append(i)
    df1[i]=data_train[i]
  else:
    df2[i]=data_train[i]
    if(np.dtype(data_train[i])=='int64'):
      r.append(i)

In [ ]:
df2.info()

### ***Using Iterative Imputer***

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(df2)
imputed = pd.DataFrame(imp_mean.transform(df2))
imputed.columns = df2.columns
df2=imputed
df2

In [ ]:
df2.info()
df2.head()

In [ ]:
df3=pd.get_dummies(df1)

In [ ]:
df3.info()
df3.head()

In [ ]:
data_train=df3.join(df2)
data_train.info()

In [ ]:
sns.heatmap(data=data_train,cmap='icefire')

In [ ]:
data_train.head()

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train.info()

In [ ]:
target=data_train['SalePrice']

In [ ]:
sns.heatmap(data=data_train.isnull(),cmap="tab20")

In [ ]:
data_train.info()

In [ ]:
data_test.info()

In [ ]:
df5=pd.DataFrame()
df5['Id']=data_test['Id']
data_test=data_test.drop(columns=['Id','Alley','PoolQC','Fence','MiscFeature'])

In [ ]:
df1=pd.DataFrame()
df2=pd.DataFrame()
l=[]
r=[]
for i in data_test:
  if(np.dtype(data_test[i])=='object'):
    l.append(i)
    df1[i]=data_test[i]
  else:
    df2[i]=data_test[i]
    if(np.dtype(data_test[i])=='int64'):
      r.append(i)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(df2)
imputed = pd.DataFrame(imp_mean.transform(df2))
imputed.columns = df2.columns
df2=imputed
df2

In [ ]:
df3=pd.get_dummies(df1)
data_test=df3.join(df2)
data_test.info()

In [ ]:
sns.heatmap(data=data_test,cmap='viridis')

In [ ]:
data_test.isnull().sum()

In [ ]:
r=[]
for i in data_test:
  r.append(i)

In [ ]:
data_test.info()

In [ ]:
r.append("SalePrice")

In [ ]:
data_train=data_train[r]

## ***Feature Selection***

In [ ]:
l=abs(data_train.corr()["SalePrice"][abs(data_train.corr()["SalePrice"])>0.05].drop('SalePrice')).index.tolist()
data_train=data_train[l]
data_train.to_csv("imputed.csv")
data_train=data_train.join(target)
data_train.info()

In [ ]:
data_test=data_test[l]

In [ ]:
data_test.info()
data_train.info()

In [ ]:
x = data_train.drop(['SalePrice'], axis=1)
y = data_train['SalePrice']
z = data_test

# **Modeling**

# ***XGBoost***

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=4,learning_rate=0.11,colsample_bytree=1,gamma=0.01)
xgb.fit(x, y)
xgb_pred = xgb.predict(z)
final_xgb=pd.DataFrame()
final_xgb['SalePrice'] = xgb_pred

In [ ]:
final=pd.DataFrame()
final['Id'] = df5['Id']

In [ ]:
final=final.join(final_xgb)

In [ ]:
final.tail()

In [ ]:
#final.to_csv("Royal_Xgboost.csv",index=False)

In [ ]:
np.dtype(final['SalePrice'])

In [ ]:
final.info()
final.head()

# CatBoost

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor
cat = CatBoostRegressor(iterations=2000)
cat.fit(x, y)
pred_cat = cat.predict(z)
final_cat=pd.DataFrame()
final_cat['SalePrice'] = pred_cat

In [ ]:
final=pd.DataFrame()
final['Id'] = df5['Id']

In [ ]:
final=final.join(final_cat)

In [ ]:
#final.to_csv("Royal_CatBoost.csv",index=False)

In [ ]:
final.info()
final.head()